# Advanced Analytics and Applications
## Analytics Anacondas 

### Data Collection and Preparation 

In [17]:
# import relevant libaries 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import glob 
from vincenty import vincenty
import datetime

In [2]:
# take a look into the data and its structure 
df = pd.read_csv("Data/2019/01/positions_nextbike_bonn_2019_01_28.csv", sep=',')


In [3]:
# explore columns and its meanings 
df.columns

Index(['p_rack_locks', 'p_bike_racks', 'b_state', 'p_spot', 'b_active',
       'p_booked_bikes', 'p_place_type', 'datetime', 'b_number', 'trip',
       'p_uid', 'p_bikes', 'p_lat', 'b_electric_lock', 'b_bike_type', 'p_name',
       'p_free_racks', 'p_address', 'b_lock_types', 'city', 'p_number',
       'b_pedelec_battery', 'p_lng', 'b_boardcomputer', 'p_maintenance',
       'p_terminal_type', 'p_bike', 'p_bike_types'],
      dtype='object')

In [4]:
# TODO table of columns with its meanings 

In [5]:
# show dataframe with columns that might be useful for further analyses and the computation of trip data
df = df[["b_active","p_place_type","datetime","b_number","trip","p_uid","p_bikes","p_lat","p_lng","p_name","city","p_number","p_bike","p_bike_types"]]


In [6]:
df.head(20)

,b_active,p_place_type,datetime,b_number,trip,p_uid,p_bikes,p_lat,p_lng,p_name,city,p_number,p_bike,p_bike_types
0,True,12,2019-01-28 00:00:00,44728,first,12238528,1,50.751664,7.119859,BIKE 44728,bonn,0,True,"{""71"": 1}"
1,True,12,2019-01-28 23:59:00,44728,last,12292496,1,50.751783,7.119811,BIKE 44728,bonn,0,True,"{""71"": 1}"
2,True,12,2019-01-28 00:00:00,44658,first,12266270,1,50.748038,7.056917,BIKE 44658,bonn,0,True,"{""71"": 1}"
3,True,12,2019-01-28 23:59:00,44658,last,12266270,1,50.748084,7.056791,BIKE 44658,bonn,0,True,"{""71"": 1}"
4,True,12,2019-01-28 00:00:00,44929,first,12270065,1,50.719922,7.095922,BIKE 44929,bonn,0,True,"{""71"": 1}"
5,True,12,2019-01-28 23:59:00,44929,last,12270065,1,50.719921,7.095809,BIKE 44929,bonn,0,True,"{""71"": 1}"
6,True,12,2019-01-28 00:00:00,44607,first,12271247,1,50.749891,7.043951,BIKE 44607,bonn,0,True,"{""71"": 1}"
7,True,12,2019-01-28 23:59:00,44607,last,12307708,1,50.750058,7.043390,BIKE 44607,bonn,0,True,"{""71"": 1}"
8,True,12,2019-01-28 00:00:00,44650,first,12277788,1,50.725752,7.034684,BIKE 44650,bonn,0,True,"{""71"": 1}"
9,True,12,2019-01-28 23:59:00,44650,last,12277788,1,50.725879,7.034632,BIKE 44650,bonn,0,True,"{""71"": 1}"


In [7]:
df["trip"].unique()

array(['first', 'last', 'start', 'end'], dtype=object)

There are four different values in the trip column [first, last, start, end]. 
At least two values are required to define whether the data set belongs to the starting point or the end of the trip. This means that one trip is represented in two rows in the dataframe. One of the rows contains the values at the starting point (i.e. datetime, start position) and the other row contains the values at the ending point of the trip. 


###### Why are there four values in the trip column? 

Let's have a deeper look in the dataframe and the trip column.

In [8]:
df[(df["trip"] == "first") | (df["trip"] =="last")].head(20)

,b_active,p_place_type,datetime,b_number,trip,p_uid,p_bikes,p_lat,p_lng,p_name,city,p_number,p_bike,p_bike_types
0,True,12,2019-01-28 00:00:00,44728,first,12238528,1,50.751664,7.119859,BIKE 44728,bonn,0,True,"{""71"": 1}"
1,True,12,2019-01-28 23:59:00,44728,last,12292496,1,50.751783,7.119811,BIKE 44728,bonn,0,True,"{""71"": 1}"
2,True,12,2019-01-28 00:00:00,44658,first,12266270,1,50.748038,7.056917,BIKE 44658,bonn,0,True,"{""71"": 1}"
3,True,12,2019-01-28 23:59:00,44658,last,12266270,1,50.748084,7.056791,BIKE 44658,bonn,0,True,"{""71"": 1}"
4,True,12,2019-01-28 00:00:00,44929,first,12270065,1,50.719922,7.095922,BIKE 44929,bonn,0,True,"{""71"": 1}"
5,True,12,2019-01-28 23:59:00,44929,last,12270065,1,50.719921,7.095809,BIKE 44929,bonn,0,True,"{""71"": 1}"
6,True,12,2019-01-28 00:00:00,44607,first,12271247,1,50.749891,7.043951,BIKE 44607,bonn,0,True,"{""71"": 1}"
7,True,12,2019-01-28 23:59:00,44607,last,12307708,1,50.750058,7.043390,BIKE 44607,bonn,0,True,"{""71"": 1}"
8,True,12,2019-01-28 00:00:00,44650,first,12277788,1,50.725752,7.034684,BIKE 44650,bonn,0,True,"{""71"": 1}"
9,True,12,2019-01-28 23:59:00,44650,last,12277788,1,50.725879,7.034632,BIKE 44650,bonn,0,True,"{""71"": 1}"


In this filtered dataframe above it gets clear that the examples with the values **first** and **last** in the trip column don't make much sense. Most of the trips in this dataframe have an unlikely long trip duration. The start time of a trip is almost always at 0 AM and the end time of a trip is at 23:59 PM. 
Furthermore the start and the end positions of one trip are nearly the same. 

It could be measurement errors or other data recording procedures. <br> 
These datasets can be disregarded for the next steps, because they aren't suitable for further analyses and they don't represent a real trip.  

Another column which becomes clear is the **p_number**. If the p_number is eqaul to zero then the p_name has a value that contains 'BIKE'. IF the p_number isn't equal to zero then there exists a name for the positon. This indicates that the trip starts or ends at an official bike station. 

In [9]:
# datasets where the duration of a trip is much more meaningful  
df[(df["trip"] == "start") | (df["trip"] =="end")].head(20)

,b_active,p_place_type,datetime,b_number,trip,p_uid,p_bikes,p_lat,p_lng,p_name,city,p_number,p_bike,p_bike_types
11,True,12,2019-01-28 23:23:00,44647,start,12262294,1,50.739120,7.073766,BIKE 44647,bonn,0,True,"{""71"": 1}"
12,True,12,2019-01-28 23:29:00,44647,end,12308310,1,50.742764,7.073442,BIKE 44647,bonn,0,True,"{""71"": 1}"
27,True,12,2019-01-28 09:37:00,44795,start,12233021,1,50.691034,7.130670,BIKE 44795,bonn,0,True,"{""71"": 1}"
28,True,12,2019-01-28 09:47:00,44795,end,12292170,1,50.686653,7.159832,BIKE 44795,bonn,0,True,"{""71"": 1}"
35,True,12,2019-01-28 08:47:00,44719,start,12276651,1,50.743868,7.079610,BIKE 44719,bonn,0,True,"{""71"": 1}"
36,True,12,2019-01-28 09:00:00,44719,end,12291350,1,50.724698,7.087712,BIKE 44719,bonn,0,True,"{""71"": 1}"
37,True,12,2019-01-28 09:07:00,44719,start,12291350,1,50.724698,7.087712,BIKE 44719,bonn,0,True,"{""71"": 1}"
38,True,12,2019-01-28 09:39:00,44719,end,12292023,1,50.724831,7.087654,BIKE 44719,bonn,0,True,"{""71"": 1}"
39,True,12,2019-01-28 16:43:00,44719,start,12294951,1,50.724819,7.087713,BIKE 44719,bonn,0,True,"{""71"": 1}"
40,True,12,2019-01-28 16:52:00,44719,end,12300675,1,50.718751,7.107172,BIKE 44719,bonn,0,True,"{""71"": 1}"


A trip consists of two successive datasets. The first contains the values at the start and the second dataset contains the values at the ending point of the trip. In the next steps we can join these datasets to one dataset, so that a trip is represented in just one dataset with features about the start and end time and the start and end location as well. 

In [10]:
# read in the availabe data for every month and day 
df = pd.DataFrame()

#read in files for each month 
for i in range(1,13): 
    if i <10: 
        #read in files for each day
        for i in glob.glob("Data/2019/0" + str(i) +"/*.csv"):     
            df_1 = pd.read_csv(i, sep=',')
            df = pd.concat([df, df_1],axis=0)    # concatenate to one dateframe     
    else: 
        #read in files for each day
        for i in glob.glob("Data/2019/" + str(i) +"/*.csv"):     
            df_1 = pd.read_csv(i, sep=',')
            df = pd.concat([df, df_1],axis=0)    # concatenate to one dateframe     

df.reset_index(drop=True,inplace=True)            


C:\Users\Emre\Anaconda3\envs\PDS20\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
# function that gets the raw data and returns the final trip data 
def get_df_final_trips(p_df): 
    
    # discard values 'first' & 'last' in trip column 
    p_df = p_df[(p_df["trip"] == "start") | (p_df["trip"]=="end")]
    
    # select useful columns for further analyses 
    # if neccessary we can add some more columns here 
    p_df = p_df[["p_place_type","datetime","b_number","trip","p_uid","p_bikes","p_lat","p_lng",
            "p_name","p_number","p_bike"]]
    
    # split data into two dataframes 
    df_start = p_df[(p_df["trip"] == "start")] 
    df_end = p_df[(p_df["trip"] == "end")]

    # reset indexes to get current indexes as a further column 
    df_start.reset_index(inplace=True)
    df_end.reset_index(inplace=True)

    # rename columns for clarity after merging
    df_start.rename(columns={"index":"index_start",
                         "p_place_type":"p_place_type_start",
                         "datetime":"datetime_start",
                         "b_number":"b_number_start",
                         "trip":"trip_start",
                         "p_uid":"p_uid_start",
                         "p_bikes":"p_bikes",
                         "p_lat":"latitude_start",
                         "p_lng":"longitude_start",
                         "p_name":"p_name_start",
                         "p_number":"p_number_start",
                         "p_bike":"p_bike_start"},                         
                inplace=True)

    df_end.rename(columns={"index":"index_end",
                         "p_place_type":"p_place_type_end",
                         "datetime":"datetime_end",
                         "b_number":"b_number_end",
                         "trip":"trip_end",
                         "p_uid":"p_uid_end",
                         "p_bikes":"p_bikes",
                         "p_lat":"latitude_end",
                         "p_lng":"longitude_end",
                         "p_name":"p_name_end",
                         "p_number":"p_number_end",
                         "p_bike":"p_bike_end"},
                inplace=True)

    # a trip consists of two successive datasets
    # decrement index_end for merging with index_start
    df_end["index_end"] = df_end["index_end"]-1
    
    # merge the two dataframes to one 
    # the result is the dataframe that contains one trip per row
    final_trips = pd.merge(df_start,df_end,left_on="index_start", right_on="index_end")
    
    return final_trips

In [12]:
# save final trip data
final_trips = get_df_final_trips(df)

C:\Users\Emre\Anaconda3\envs\PDS20\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Emre\Anaconda3\envs\PDS20\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
# would not make sense, if bike id is not equal at start and end location
final_trips[final_trips["b_number_start"] != final_trips["b_number_start"]]

,index_start,p_place_type_start,datetime_start,b_number_start,trip_start,p_uid_start,p_bikes_x,latitude_start,longitude_start,p_name_start,...,datetime_end,b_number_end,trip_end,p_uid_end,p_bikes_y,latitude_end,longitude_end,p_name_end,p_number_end,p_bike_end


In [14]:
# check missing values 
final_trips.isna().any(axis=0)

index_start           False
p_place_type_start    False
datetime_start        False
b_number_start        False
trip_start            False
p_uid_start           False
p_bikes_x             False
latitude_start        False
longitude_start       False
p_name_start          False
p_number_start         True
p_bike_start          False
index_end             False
p_place_type_end      False
datetime_end          False
b_number_end          False
trip_end              False
p_uid_end             False
p_bikes_y             False
latitude_end          False
longitude_end         False
p_name_end            False
p_number_end           True
p_bike_end            False
dtype: bool

In [15]:
# converting objects to datetimes
final_trips["datetime_start"] = pd.to_datetime(final_trips["datetime_start"])
final_trips["datetime_end"] = pd.to_datetime(final_trips["datetime_end"])

# adding the trip duration with the difference of start and end time
final_trips["trip_duration"] = final_trips["datetime_end"] - final_trips["datetime_start"]

#converting timedelta to numeric and format in minutes 
final_trips["trip_duration"] = pd.to_numeric(final_trips["trip_duration"] / 60000000000)

## adding the weekday of the start time of a trip; stored in integers (0: monday, 6:sunday)
final_trips['weekday'] = final_trips['datetime_start'].dt.dayofweek

# adding the distance between start and end position
final_trips["distance"] = final_trips[["latitude_start","longitude_start","latitude_end","longitude_end"]].apply(
    lambda x: vincenty([x[0], x[1]],
                       [x[2], x[3]]),axis=1)

In [19]:
# function which returns True for saturday and sunday; otherwise it returns False
def isWeekend(index_of_day): 
    if index_of_day > 4: 
        return True 
    else: 
        return False

In [20]:
# adding new boolean column "weekend"    
final_trips["weekend"] = final_trips["weekday"].apply(lambda x: isWeekend(x))

# transform column "datatime_start" into several columns 
final_trips["day"] = final_trips["datetime_start"].apply(lambda x: x.day)
final_trips["month"] = final_trips["datetime_start"].apply(lambda x: x.month)
final_trips["hour"] = final_trips["datetime_start"].apply(lambda x: x.hour)

In [21]:
df.isna().any(axis=0)

p_rack_locks            False
p_bike_racks            False
b_state                 False
p_spot                  False
b_active                False
p_booked_bikes          False
p_place_type            False
datetime                False
b_number                False
trip                    False
p_uid                   False
p_bikes                 False
p_lat                   False
b_electric_lock         False
b_bike_type             False
p_name                  False
p_free_racks            False
p_address                True
b_lock_types            False
city                    False
p_number                 True
b_pedelec_battery        True
p_lng                   False
b_boardcomputer         False
p_maintenance           False
p_terminal_type          True
p_bike                  False
p_bike_types            False
b_battery_pack           True
p_special_racks          True
p_free_special_racks     True
dtype: bool

In [22]:
final_trips.to_csv("Data/final_trips.csv")